# Corruption analysis
Today is possible for us to readily and constantly check information online, over the years more and more people got access to mobile devices and this medium became the main for information exchange and gathering. We can imagine how this can influency society, but we're not going to study how people are interacting or being influenced, we're going to analyze the relations between the Internet users...

## About the data

We used three datasets*:
- Corruption Perception Index (CPI)
- Number Of Internet Users (# people with internet access)
- Number Of Journalists Killed

**All the datasets were downloaded from [Gap Minder](https://www.gapminder.org)*

## Importing modules and data

First we need to import the modules that we're going to use for wrangling the data. Also import the three datasets that's going to be cleaned before the analysis.

In [99]:
import pandas as pd
import numpy as np

df_internet = pd.read_csv('data/internet_users.csv')
df_killings = pd.read_csv('data/num_of_journalists_killed.csv')
df_cpi = pd.read_csv('data/corruption_perception_index_cpi.csv')

In [100]:
# Let's create a list with the datasets for easier iteration
dfs = [df_internet, df_killings, df_cpi]

## Assessing the data

Let's analyze the dataframes to see if we have any data missing or need to adequate any data type.

In [101]:
for d in dfs:
    print('Dataframe Shape: ', d.shape)
    print(d.head())
    print(d.info())

Dataframe Shape:  (193, 29)
       country  1990  1991  1992  1993     1994     1995     1996     1997  \
0  Afghanistan   NaN   NaN   NaN   NaN      NaN      NaN      NaN      NaN   
1      Albania   NaN   NaN   NaN   NaN      NaN  0.01120  0.03220  0.04860   
2      Algeria   NaN   NaN   NaN   NaN  0.00036  0.00177  0.00174  0.01030   
3      Andorra   NaN   NaN   NaN   NaN      NaN      NaN  1.53000  3.05000   
4       Angola   NaN   NaN   NaN   NaN      NaN      NaN  0.00078  0.00567   

     1998  ...   2008   2009  2010  2011   2012  2013  2014   2015  2016  2017  
0     NaN  ...   1.84   3.55   4.0   5.0   5.45   5.9   7.0   8.26  10.6   NaN  
1  0.0650  ...  23.90  41.20  45.0  49.0  54.70  57.2  60.1  63.30  66.4   NaN  
2  0.0202  ...  10.20  11.20  12.5  14.9  18.20  22.5  29.5  38.20  42.9   NaN  
3  6.8900  ...  70.00  78.50  81.0  81.0  86.40  94.0  95.9  96.90  97.9   NaN  
4  0.0185  ...   1.90   2.30   2.8   3.1   6.50   8.9  10.2  12.40  13.0   NaN  

[5 rows x 29 col

### Assessing - Fill NaN and correct type

We can observe in the log above that we have some missing numbers and one column with `int` data type. Let's fix that with the code below.

In [102]:
# Fill NaN with 0 considering that we only have float or int values that are null
# And fix data type, but skipping the first column
for d in dfs:
    d.fillna(0, inplace=True)
    d = d[d.columns[1:]].astype(float)

## Cleaning the datasets

After assessing our data, let's drop some columns in our dataframes for mantain only the columns that match between those dataframes. We're going to remove the year of 2017 also, because some incomplete data.

In [103]:
# Drop columns in Internet_users dataframe
dfs[0].drop(dfs[0].columns.to_series()['1990':'2011'], axis=1, inplace=True)
dfs[0].drop('2017', axis=1, inplace=True)

# Drop columns in Journalists_killed dataframe
dfs[1].drop(dfs[1].columns.to_series()['1992':'2011'], axis=1, inplace=True)
dfs[1].drop(['2017', '2018'], axis=1, inplace=True)

# Drop columns in CPI dataframe
dfs[2].drop('2017', axis=1, inplace=True)

## Combining the data and create a unique dataset

Now that our data is with the same number of columns and the missing data are filled we're going to combine the datasets following the [Tidy Data principles](https://cfss.uchicago.edu/notes/tidy-data/).

![Tidy Data dataset](https://r4ds.had.co.nz/images/tidy-1.png)

In [104]:
# Let's create a copy of our dataframes' lists
temp = dfs.copy()
temp[0].head()

,country,2012,2013,2014,2015,2016
0,Afghanistan,5.45,5.9,7.0,8.26,10.6
1,Albania,54.70,57.2,60.1,63.30,66.4
2,Algeria,18.20,22.5,29.5,38.20,42.9
3,Andorra,86.40,94.0,95.9,96.90,97.9
4,Angola,6.50,8.9,10.2,12.40,13.0


### Pivoting tables

We're going to pivot the year columns and create one unique column called `name` and one column for the `value`, this keep our data organized as described in the image above, with the columns as variables and our observations as rows.

In [105]:
for i in range(0,len(temp)):
    temp[i]=temp[i].melt(id_vars='country', value_vars=temp[i].columns[1:7])
    temp[i].rename(columns={'variable': 'year'}, inplace=True)

In [106]:
# After pivoting our column for the value needs to be renamed before merging the datasets
temp[0].rename(columns={'value': 'internet_users'}, inplace=True)
temp[1].rename(columns={'value': 'journalists_killed'}, inplace=True)
temp[2].rename(columns={'value': 'cpi'}, inplace=True)

In [107]:
# Before merging our data, we need to exclude the rows for that countries that none journalist was killed
group_killed = temp[1].groupby('country').sum()
none_killed = group_killed[group_killed['journalists_killed'] == 0].index

temp[1] = temp[1].set_index('country').drop(list(none_killed), axis=0)

In [108]:
# Now we simply merge the datasets in the indexes country and year
t1 = temp[0].merge(temp[1], on=['country', 'year'], how='inner')
t2 = t1.merge(temp[2], on=['country', 'year'], how='inner')

In [109]:
# For complete our transformations we use pivot_table to define our new indexes
ind = list(t2.columns[0:2])
col = list(t2.columns[2:])

t3 = t2.pivot_table(index=ind, values=col)
t3.head()

cpi  internet_users  journalists_killed
country     year                                          
Afghanistan 2012   8.0            5.45                 0.0
            2013   8.0            5.90                 0.0
            2014  12.0            7.00                 3.0
            2015  11.0            8.26                 0.0
            2016  15.0           10.60                 4.0

## Save new dataset

Now that we finished the transformations needed we can save the new compiled and cleaned dataset to a new file!

In [114]:
# Save to CSV new dataset
t3.to_csv('data/cleaned_df_cpi-killings-internet.csv')